In [ ]:
%pip install auto-sklearn

     |████████████████████████████████| 6.4 MB 5.5 MB/s 
     |████████████████████████████████| 38.1 MB 1.8 MB/s 
     |████████████████████████████████| 22.3 MB 7.5 MB/s 
     |████████████████████████████████| 1.1 MB 27.1 MB/s 
     |████████████████████████████████| 837 kB 33.8 MB/s 
     |████████████████████████████████| 4.3 MB 9.3 MB/s 
     |████████████████████████████████| 4.0 MB 17.9 MB/s 
     |████████████████████████████████| 167 kB 32.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 32.6 MB/s 
     |████████████████████████████████| 136 kB 51.7 MB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 
     |████████████████████████████████| 260 kB 40.4 MB/s 
  Created wheel for auto-sklearn: filename=auto_sklearn-0.14.6-py3-none-any.whl size=6602835 sha256=e2a014c91c27ec01371f2f324ed8b73f0c260a9d5e74555216f35493e61e571e
  Stored in direc

In [ ]:
import sklearn.datasets
import sklearn.model_selection
import autosklearn.classification
import autosklearn.metrics
import pandas as pd

In [ ]:
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id = name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  return (X_train, X_test, y_train, y_test)

In [ ]:
X_train, X_test, y_train, y_test = dataSetup(41163)
X_train.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V1991,V1992,V1993,V1994,V1995,V1996,V1997,V1998,V1999,V2000
4901,4874,4877,4884,4886,4879,4878,4880,4888,4881,4882,...,4880,4876,4889,4890,4878,4879,4890,4885,4885,4880
4375,4353,4353,4360,4361,4358,4355,4359,4366,4363,4359,...,4359,4356,4364,4364,4356,4358,4369,4360,4361,4358
6698,6647,6652,6666,6667,6660,6660,6667,6665,6668,6658,...,6657,6651,6672,6675,6658,6662,6670,6671,6672,6656
9805,9687,9715,9715,9721,9716,9717,9726,9723,9729,9714,...,9714,9717,9760,9745,9703,9734,9750,9745,9745,9721
1101,1099,1100,1100,1101,1101,1099,1099,1100,1101,1101,...,1099,1101,1100,1101,1101,1100,1100,1100,1101,1100


In [ ]:
estimator_askl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=3600,seed=42,resampling_strategy='cv',)
estimator_askl.fit(X_train, y_train, dataset_name='car')
prediction = estimator_askl.predict(X_test)

In [ ]:
(prediction == y_test).sum()/len(prediction)

0.192

In [ ]:
print(estimator_askl.sprint_statistics())

auto-sklearn results:
  Dataset name: car
  Metric: accuracy
  Best validation score: 0.215733
  Number of target algorithm runs: 22
  Number of successful target algorithm runs: 3
  Number of crashed target algorithm runs: 2
  Number of target algorithms that exceeded the time limit: 9
  Number of target algorithms that exceeded the memory limit: 8



In [ ]:
from pprint import pprint
pprint(estimator_askl.show_models(), indent=4)

{   4: {   'cost': 0.7842666666666667,
           'ensemble_weight': 0.44,
           'estimators': [   {   'balancing': Balancing(random_state=42, strategy='weighting'),
                                 'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f50d085eb90>,
                                 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f50d12a8d10>,
                                 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f50ac202fd0>,
                                 'sklearn_classifier': MLPClassifier(activation='tanh', alpha=0.00021148999718383549, beta_1=0.999,
              beta_2=0.9, hidden_layer_sizes=(113, 113, 113),
              learning_rate_init=0.0007452270241186694, max_iter=64,
              n_iter_no_change=32, random_state=42, validation_fraction=0.0,
              verbose=0, warm

In [ ]:
for run_key in estimator_askl.automl_.runhistory_.data:
    print('#########')
    print(run_key)
    print(estimator_askl.automl_.runhistory_.data[run_key])

#########
RunKey(config_id=1, instance_id='{"task_id": "car"}', seed=0, budget=0.0)
RunValue(cost=1.0, time=360.120178937912, status=<StatusType.TIMEOUT: 2>, starttime=1651523157.5610538, endtime=1651523518.7164145, additional_info={'error': 'Timeout', 'configuration_origin': 'Initial design'})
#########
RunKey(config_id=2, instance_id='{"task_id": "car"}', seed=0, budget=0.0)
RunValue(cost=1.0, time=360.0373544692993, status=<StatusType.TIMEOUT: 2>, starttime=1651523518.8775756, endtime=1651523879.948918, additional_info={'error': 'Timeout', 'configuration_origin': 'Initial design'})
#########
RunKey(config_id=3, instance_id='{"task_id": "car"}', seed=0, budget=0.0)
RunValue(cost=0.7842666666666667, time=182.23550176620483, status=<StatusType.SUCCESS: 1>, starttime=1651523880.10284, endtime=1651524062.3748753, additional_info={'duration': 182.08732628822327, 'num_run': 4, 'train_loss': 0.7121333333333333, 'configuration_origin': 'Initial design'})
#########
RunKey(config_id=4, instanc

In [ ]:
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, prediction))

Accuracy score 0.192
